<a href="https://colab.research.google.com/github/mosfet-sudo/ML-Project-G75/blob/main/90049_A2_G75_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

note: the github must be public in order for us to view 'live updates' of any changes made to this notebook

# Mount google drive
run the code below to mount your google drive to this colab notebook for however long you need to be running the code for
*Please accept Amelia's invitation to share the drive folder 'IML_G75_Colab_Notebooks*

In [ ]:
from google.colab import drive # To read in data from google drive (delete later?)
drive.mount('/content/drive') # to read in data from google drive (delete later?)

# Simple Data Processing
(Done by Elijah)
Each Dataset was checked for duplicate entries, and then specific numeric values were normalized. In addition to this there was some minor text processing of the NASA dataset where the text files were tranferred into csv format. Furthermore adjustments were made to the metrodataset where all data points of each minute were averaged in order to make the dataset smaller. In particular for values that were binary (i.e) 1 or 0 if the average was strictly greater than 0.5 then it was set to 1 otherwise it was set to 0. This averaging should reduce noise and reduce the affect of anomalous outliers.

## Metro-PT3

In [ ]:
# The processed dataset has been uploaded to the drive already, might not be advisable to run this code (raw data file is too large)
import pandas as pd
import numpy as np
import json

## averaging function for new dataset that preserves when errors occur, n must be even n=6 for averaging every minute
## takes the data  frame the number of samples to bunch together and average and the indexes that are we are applying the averaging off.
## the inputs are the data frame, an integer n, and 2 arrays of numeric indexes it returns the new (smaller) averaged dataframe
def timeAvg(df, n, tempindexes, elec_temp_indexes):
    m = n/2
    m = int(m)

    df_empty = df.copy(deep=True).iloc[0:0]
    avg_atpoints = []
    max_atpoints = []
    for i in tempindexes:
        print(i)
        avg_atpoints = []
        temp_series = df.iloc[:,i]
        numrows = int(temp_series.shape[0])
        for j in range(m, int(numrows-m),n):
            avg_atpoints.append(temp_series.iloc[j-m:j+m].mean()) #here we take the average
        df_empty.iloc[:,i] = avg_atpoints
    for i in elec_temp_indexes:
        print(i)
        max_atpoints = []
        temp_series = df.iloc[:,i]
        numrows = int(temp_series.shape[0])
        for j in range(m, int(numrows-m),n):
            avg = temp_series.iloc[j-m:j+m].mean()  #we use the average to decide what to assign to this datapoint
            if avg > 0.5:
                max_atpoints.append(1)
            else:
                max_atpoints.append(0)
        df_empty.iloc[:,i] = max_atpoints
    df_empty.iloc[:,0] = list(range(1,df_empty.shape[0]+1))
    return df_empty


# df = pd.read_csv("C:/Users/ej/PycharmProjects/Assignment2/.venv/Scripts/MetroPT3(AirCompressor).csv")
df = pd.read_csv('/content/drive/My Drive/IML_G75_Colab_Notebooks/')
#Duplicate checking

df = df.drop(columns = ["timestamp"])
num_columns = df.shape[1]
num_rows = df.shape[0]
duplicateddf=df.duplicated()
no_duplicate_df = df.loc[duplicateddf==False]


no_duplicate_df.rename(columns={"Unnamed: 0": "Time Elapased [seconds]"}, inplace=True)
print(no_duplicate_df.columns)

# Normalisation
# Find the indexes of columns which have values we want to normalize
temp_indexes = [no_duplicate_df.columns.get_loc("Time Elapased [seconds]")] ##SHOULD WE NORMALIsE THIS?
temp_indexes.append(no_duplicate_df.columns.get_loc("TP2"))
temp_indexes.append(no_duplicate_df.columns.get_loc("TP3"))
temp_indexes.append(no_duplicate_df.columns.get_loc("H1"))
temp_indexes.append(no_duplicate_df.columns.get_loc("DV_pressure"))
temp_indexes.append(no_duplicate_df.columns.get_loc("Reservoirs"))
temp_indexes.append(no_duplicate_df.columns.get_loc("Oil_temperature"))
temp_indexes.append(no_duplicate_df.columns.get_loc("Motor_current"))


# BOTH RPM AND TOOLWEAR HAVE ISSUES WITH CERTAIN DTYPES BEING INT DUE TO WHOLE NUMBERS IN DATA
no_duplicate_df["Time Elapased [seconds]"] = no_duplicate_df["Time Elapased [seconds]"].astype(float)

#for each index normalize the values and for each row set the value to the normalized value
for i in temp_indexes:
    temp_series = no_duplicate_df.iloc[:,i]
    series_max = temp_series.max()
    series_min = temp_series.min()
    series_range = temp_series.max() - temp_series.min()

    for j in range(temp_series.shape[0]):

        no_duplicate_df.iat[j,i] = (no_duplicate_df.iat[j,i]-series_min)/series_range



## new temp indexes for finding columns of electrical signal data
elec_temp_indexes = [no_duplicate_df.columns.get_loc("COMP")]
elec_temp_indexes.append(no_duplicate_df.columns.get_loc("DV_eletric"))
elec_temp_indexes.append(no_duplicate_df.columns.get_loc("Towers"))
elec_temp_indexes.append(no_duplicate_df.columns.get_loc("MPG"))
elec_temp_indexes.append(no_duplicate_df.columns.get_loc("LPS"))
elec_temp_indexes.append(no_duplicate_df.columns.get_loc("Pressure_switch"))
elec_temp_indexes.append(no_duplicate_df.columns.get_loc("Oil_level"))
elec_temp_indexes.append(no_duplicate_df.columns.get_loc("Caudal_impulses"))



no_dupe_normalized_df = no_duplicate_df
no_dupe_normalized_df_avg = timeAvg(no_dupe_normalized_df,6,temp_indexes,elec_temp_indexes)

no_dupe_normalized_df_avg.to_csv("METROPT3_AVGMINUTEPROCESSED.csv")

## NASA

In [ ]:
import pandas as pd
import numpy as np

#Method for parsing text files into dataframe, takes a string input and returns a dataframe object
def parseTxt2Dataframe(String):
    with open(String, 'r', encoding='utf-8') as file:
        lines = []
        for line in file:
            newline = line.strip()
            newline = newline.split()
            lines.append(newline)
        new_df = pd.DataFrame(lines, columns=['Unit Number', 'Time in cycles',
                                  'Operational Setting 1', 'Operational Setting 2','Operational Setting 3',
                                  'T2','T24','T30','T50','P2','P15','P30','Nf','Nc','epr','Ps30','phi','NRf','NRc',
                                  'BPR','farB','htBleed','Nf_dmd','PCNfr_dmd','W31','W32'])
        num_cols = new_df.shape[1]
        for i in range(num_cols):
            new_df.iloc[:,i] = pd.to_numeric(new_df.iloc[:,i])


    return new_df

#normalizes the data, and checks for duplicates, takes dataframe object as input and returns a dataframe
def normalize_and_checkdupe(df):
    checkdf = df.drop(columns=['Unit Number','Time in cycles'])
    duplicateddf = checkdf.duplicated()
    no_dupe_df = df.loc[duplicateddf==False]
    num_cols = no_dupe_df.shape[1]
    num_rows = no_dupe_df.shape[0]
    for i in range(2,num_cols):
        max_val = no_dupe_df.iloc[:,i].max()
        min_val = no_dupe_df.iloc[:,i].min()
        val_range = max_val - min_val
        for j in range(num_rows):
            if val_range != 0:
                no_dupe_df.iat[j, i] = (no_dupe_df.iat[j, i] - min_val) / val_range
            else:
                no_dupe_df.iat[j, i] = -1

    return no_dupe_df


# parsing each text file into a dataframe
df_test1 = parseTxt2Dataframe('/content/drive/My Drive/IML_G75_Colab_Notebooks/CMaps_NASA/test_FD001.txt')
df_test2 = parseTxt2Dataframe('/content/drive/My Drive/IML_G75_Colab_Notebooks/CMaps_NASA/test_FD002.txt')
df_test3 = parseTxt2Dataframe('/content/drive/My Drive/IML_G75_Colab_Notebooks/CMaps_NASA/test_FD003.txt')
df_test4 = parseTxt2Dataframe('/content/drive/My Drive/IML_G75_Colab_Notebooks/CMaps_NASA/test_FD004.txt')
df_train1 = parseTxt2Dataframe('/content/drive/My Drive/IML_G75_Colab_Notebooks/CMaps_NASA/train_FD001.txt')
df_train2 = parseTxt2Dataframe('/content/drive/My Drive/IML_G75_Colab_Notebooks/CMaps_NASA/train_FD002.txt')
df_train3 = parseTxt2Dataframe('/content/drive/My Drive/IML_G75_Colab_Notebooks/CMaps_NASA/train_FD003.txt')
df_train4 = parseTxt2Dataframe('/content/drive/My Drive/IML_G75_Colab_Notebooks/CMaps_NASA/train_FD004.txt')



#combining the data into one dataset

df_all = pd.concat([df_test1,df_test2,df_test3,df_test4,df_train1,df_train2,df_train3,df_train4],ignore_index=True)
print(df_all.shape)
print(df_test1.shape)

## normalizing and checking for duplicates when combined
df_all_processed = normalize_and_checkdupe(df_all)

## normalizing and checking for duplicates when uncombined
df_test1_processed = normalize_and_checkdupe(df_test1)
#df_test2_processed = normalize_and_checkdupe(df_test2)
#df_test3_processed = normalize_and_checkdupe(df_test3)
#df_test4_processed = normalize_and_checkdupe(df_test4)
#df_train1_processed = normalize_and_checkdupe(df_train1)
#df_train2_processed = normalize_and_checkdupe(df_train2)
#df_train3_processed = normalize_and_checkdupe(df_train3)
#df_train4_processed = normalize_and_checkdupe(df_train4)



## testing and debug
pd.set_option('display.max_columns', None)
print(df_all_processed.shape)
print(df_all_processed.head())
print(df_test1_processed.head())
print("newline")


print("newline")
print(df_all_processed.shape)

pd.reset_option('display.max_columns')
df_all_processed.to_csv('NASAprocessed.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## PdM

In [ ]:
import pandas as pd
import numpy as np
import json
df = pd.read_csv("C:/Users/ej/PycharmProjects/Assignment2/.venv/Scripts/ai_2020.csv")
#Duplicate checking

df = df.drop(columns = ["UDI"])
num_columns = df.shape[1]
num_rows = df.shape[0]
duplicateddf=df.duplicated()
no_duplicate_df = df.loc[duplicateddf==False]

print(no_duplicate_df.columns)
#Normalization

temp_indexes = [no_duplicate_df.columns.get_loc("Air temperature [K]")]
temp_indexes.append(no_duplicate_df.columns.get_loc("Process temperature [K]"))
temp_indexes.append(no_duplicate_df.columns.get_loc("Rotational speed [rpm]")) #TURN INTO SECONDS?
temp_indexes.append(no_duplicate_df.columns.get_loc("Torque [Nm]"))
temp_indexes.append(no_duplicate_df.columns.get_loc("Tool wear [min]"))     # TURN INTO SECONDS?
# BOTH RPM AND TOOLWEAR HAVE ISSUES WITH CERTAIN DTYPES BEING INT DUE TO WHOLE NUMBERS IN DATA
no_duplicate_df["Rotational speed [rpm]"] = no_duplicate_df["Rotational speed [rpm]"].astype(float)
no_duplicate_df["Tool wear [min]"] = no_duplicate_df["Tool wear [min]"].astype(float)

for i in temp_indexes:

    temp_series = no_duplicate_df.iloc[:,i]
    series_max = temp_series.max()
    series_min = temp_series.min()
    series_range = temp_series.max() - temp_series.min()

    for j in range(temp_series.shape[0]):

        no_duplicate_df.iat[j,i] = (no_duplicate_df.iat[j,i]-series_min)/series_range


no_dupe_normalized_df = no_duplicate_df

no_dupe_normalized_df.to_csv("PredicitiveMaintenanceDatasetPROCESSED.csv")

# Advanced Pre-Processing
See sections below to get more details on how we all completed the advanced pre-processing of our datasets

## Metro PT-3
*Xianyu & Elijah*




---

**Amelia's notes from when they were mistakenly trying to complete the advanced pre-processing of the Metro PT-3 dataset:**


Advanced pre-processing focused on the following sub-question:
*   How well do electrical load (Motor_current), temperature (Oil_temperature), and output pressure (TP2, TP3) predict compressor stress or early failure?





In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
# import dataset and stuff

# make sure that simple pre-processed dataset is clean & normalised
df = df.dropna()
df = df.sort_values("Time Elapased [seconds]")

# scale numeric features for ordinal SVM
features = ["Motor_current", "Oil_temperature", "TP2", "TP3", "Reservoirs"]
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

# creating derived features
df["Temp_x_current"] = df["Oil_temperature"] * df["Motor_current"]              # thermal stress
df["Pressure_ratio"] = df["TP2"] / (df["TP3"] + 1e-6)                           # compressor efficiency
df["Reservoir_diff"] = df["Reservoirs"].diff().fillna(0)                        # system stability over time
df["Motor_delta"] = df["Motor_current"].diff().fillna(0)                        # system stability over time (pt2: electric boogaloo)

# ordinal target variable
df["HealthState"] = np.select([(df["Oil_temperature"] < 0.4) & (df["Motor_current"] < 0.4), (df["Oil_temperature"] < 0.7) & (df["Motor_current"] < 0.7),],[0, 1],default=2)



## NASA
*Xianyu & Amelia *

Advanced preprocessing involved...